# CHÚ Ý ĐỌC KĨ HƯỚNG DẪN SỬ DỤNG TRƯỚC KHI DÙNG
# ẤN TỔ HỢP PHÍM SHIFT + ENTER ĐỂ CHẠY TỪNG CELL MỘT

In [2]:
import cv2
import numpy as np 
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import matplotlib.image as image
from PIL import Image, ImageOps
from numpy import asarray
import threading

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#Color conversion BGR 2 RGB
    image.flags.writeable = False #Image is unwriteable
    results = model.process(image)#Make prediction
    image.flags.writeable = True#Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)#color conversion RGB 2 BGR
    return image, results

In [4]:
def draw_styled_landmarks(image, results):
    #Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness = 1, circle_radius =1),
                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness = 1, circle_radius =1)
                             )
    #Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness = 2, circle_radius =2)
                             )
    #Draw hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness = 2, circle_radius =2)
                             )
    #Draw hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness = 2, circle_radius =4),
                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness = 2, circle_radius =2)
                             )

In [5]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if \
        results.face_landmarks else np.zeros(468*3)
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if \
        results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if \
        results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if \
        results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [32]:
mp_holistic = mp.solutions.holistic #Holistic model
mp_drawing = mp.solutions.drawing_utils #Drawing utilities

# Khi nào đổi động tác thì chạy lại cái ô này và ô tạo folder bên dưới

In [27]:
count = 0 #Vị trí hiện tại nằm trong khoảng 0-29 của frame của folder thứ folder_num
folder_num = 0 # Vị trí hiện tại nằm trong khoảng 0-29 của folder chứa các frame
dem = 0 #Ảnh thứ bao nhiêu trong folder _image
#Tên hành động, làm hành động nào thì sửa tên cái folderName này thành hành động đó
#Ví dụ làm hành động uống nước thì folderName = 'drinkWater'
folderName = 'THANKS' 
folderForImage = folderName + '_image'
folderForNpy = folderName
IMAGE_PATH = os.path.join('Data', folderForImage)
NPY_PATH = os.path.join('Data', folderForNpy)

# Tạo 2 folder để lưu dữ liệu: folder lưu ảnh để check và folder lưu dữ liệu dưới dạng nparray 

In [28]:
os.makedirs(IMAGE_PATH)
for i in range(30):
    try:
        os.makedirs(os.path.join(NPY_PATH, str(i)))
    except Exception as exception:
        print(exception)
        pass

# Lắp camera, chuẩn bị quay 900 frames = 30s, Chú ý trong lúc quay phải tập  trung đừng để sai động tác, quay rõ ràng các điểm mặt, tay.
# KÉO XUỐNG DƯỚI CÓ CODE CHECK CAM TRƯỚC KHI QUAY THẬT ĐỂ SET GÓC CHO CHUẨN

In [33]:
numb_of_frame = 30
numb_of_sequence = 30

In [29]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for sequence_num in range(numb_of_sequence):
        # NEW LOOP
        # Loop through actions
        # Loop through video length aka sequence length
        for frame_num in range(numb_of_frame):
            npy_path = os.path.join(NPY_PATH, str(sequence_num), str(frame_num))
            img_path = os.path.join(IMAGE_PATH, '%d.jpg' % dem)
            # Read feed
            ret, frame = cap.read()

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
    #                 print(results)

            # Draw landmarks
            draw_styled_landmarks(image, results)
            # NEW Apply wait logic
            if frame_num == 0: 
                cv2.putText(image, 'PREPARE COLLECTION', (120,200), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(folderName, sequence_num), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
             # Show to screen
                cv2.imshow('OpenCV Feed', image)
                cv2.waitKey(3000)
            else: 
                cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(folderName, sequence_num), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            # Show to screen
                cv2.imshow('OpenCV Feed', image)

            # NEW Export keypoints

            keypoints = extract_keypoints(results)
            print("Save data to ", npy_path)
            np.save(npy_path, keypoints)
            cv2.imwrite(img_path, image)
            dem += 1   
    #             cv2.imwrite(os.path.join("MP_Data", "%d.jpg" % count), cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    #             keypoints = extract_keypoints(results)
    #             npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
    #             np.save(npy_path, keypoints)

            # Break gracefully
            if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

Save data to  Data\THANKS\0\0
Save data to  Data\THANKS\0\1
Save data to  Data\THANKS\0\2
Save data to  Data\THANKS\0\3
Save data to  Data\THANKS\0\4
Save data to  Data\THANKS\0\5
Save data to  Data\THANKS\0\6
Save data to  Data\THANKS\0\7
Save data to  Data\THANKS\0\8
Save data to  Data\THANKS\0\9
Save data to  Data\THANKS\0\10
Save data to  Data\THANKS\0\11
Save data to  Data\THANKS\0\12
Save data to  Data\THANKS\0\13
Save data to  Data\THANKS\0\14
Save data to  Data\THANKS\0\15
Save data to  Data\THANKS\0\16
Save data to  Data\THANKS\0\17
Save data to  Data\THANKS\0\18
Save data to  Data\THANKS\0\19
Save data to  Data\THANKS\0\20
Save data to  Data\THANKS\0\21
Save data to  Data\THANKS\0\22
Save data to  Data\THANKS\0\23
Save data to  Data\THANKS\0\24
Save data to  Data\THANKS\0\25
Save data to  Data\THANKS\0\26
Save data to  Data\THANKS\0\27
Save data to  Data\THANKS\0\28
Save data to  Data\THANKS\0\29
Save data to  Data\THANKS\1\0
Save data to  Data\THANKS\1\1
Save data to  Data\T

# CHECK GÓC TRƯỚC KHI QUAY THẬT

In [31]:
cap = cv2.VideoCapture(0)
#Set mediapipe
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        #Read feed
        ret, frame = cap.read()

        #Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        #Draw landmarks
        draw_styled_landmarks(image, results)
        #Show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #Break loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()